In [2]:
import kagglehub
import pandas as pd
import os

# Tải dataset và lấy đường dẫn
path = kagglehub.dataset_download("uciml/breast-cancer-wisconsin-data")
print("Path to dataset files:", path)

# Tìm tệp CSV
for file in os.listdir(path):
    if file.endswith('.csv'):
        csv_path = os.path.join(path, file)
        break
else:
    print("Không tìm thấy tệp CSV trong thư mục.")
    exit()

# Đọc dữ liệu
data = pd.read_csv(csv_path)

# Bước 1: Xóa cột không cần thiết ('id' và 'Unnamed: 32')
columns_to_drop = ['id', 'Unnamed: 32']
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], errors='ignore')
print("\nĐã xóa các cột không cần thiết (nếu có):", columns_to_drop)

# Bước 2: Kiểm tra và xóa giá trị trùng lặp
duplicates = data.duplicated().sum()
if duplicates > 0:
    data = data.drop_duplicates()
    print(f"Đã xóa {duplicates} hàng trùng lặp.")
else:
    print("Không có hàng trùng lặp.")

# Bước 3: Kiểm tra và xử lý giá trị thiếu
missing_values = data.isnull().sum().sum()
if missing_values > 0:
    data = data.dropna()
    print(f"Đã xóa các hàng chứa giá trị thiếu ({missing_values} giá trị).")
else:
    print("Không có giá trị thiếu.")

# Bước 4: Mã hóa cột 'diagnosis' (M = 1, B = 0)
if 'diagnosis' in data.columns:
    data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})
    print("Đã mã hóa cột 'diagnosis' thành 0 (B) và 1 (M).")

# Kiểm tra dữ liệu sau khi làm sạch
print("\nThông tin dataset sau khi làm sạch:")
print(data.info())
print("\n5 dòng đầu tiên:")
print(data.head())

# Lưu dataset sạch
data.to_csv('breast_cancer_cleaned.csv', index=False)
print("\nĐã lưu dataset sạch vào 'breast_cancer_cleaned.csv'.")

Path to dataset files: C:\Users\Admin\.cache\kagglehub\datasets\uciml\breast-cancer-wisconsin-data\versions\2

Đã xóa các cột không cần thiết (nếu có): ['id', 'Unnamed: 32']
Không có hàng trùng lặp.
Không có giá trị thiếu.
Đã mã hóa cột 'diagnosis' thành 0 (B) và 1 (M).

Thông tin dataset sau khi làm sạch:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    int64  
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    f